### 3.2.5 ResNet 残差网络
> [论文 Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

随着神经网络越来越深，保证新添加的层提升神经网络的性能并且如何设计这种网络十分重要。

假设有一类特定的神经网络架构$\mathcal{F}$，它包括学习速率和其他超参数设置。对于所有$f \in \mathcal{F}$，存在一些参数集（例如权重和偏置），这些参数可以通过在合适的数据集上进行训练而获得。

$$f^*_\mathcal{F} := \mathop{\mathrm{argmin}}_f L(\mathbf{X}, \mathbf{y}, f) \text{ subject to } f \in \mathcal{F}.$$

那么，怎样得到更近似真正$f^*$的函数呢？

唯一合理的可能性是，我们需要设计一个更强大的架构$\mathcal{F}'$。
换句话说，我们预计$f^*_{\mathcal{F}'}$比$f^*_{\mathcal{F}}$“更近似”。然而，如果$\mathcal{F}\not\subseteq \mathcal{F}'$，则无法保证新的体系“更近似”。

事实上，$f^*_{\mathcal{F}'}$可能更糟：
* 下图所示，对于非嵌套函数（non-nested function）类，较复杂的函数类并不总是向“真”函数$f^*$靠拢（复杂度由$\mathcal{F}_1$向$\mathcal{F}_6$递增）。 在下图的左边，虽然$\mathcal{F}_3$比$\mathcal{F}_1$更接近$f^*$，但$\mathcal{F}_6$却离的更远了。

* 相反对于 下图右侧的嵌套函数（nested function）类$\mathcal{F}_1 \subseteq \ldots \subseteq \mathcal{F}_6$，我们可以避免上述问题。

![](http://d2l.ai/_images/functionclasses.svg)

ResNet在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计。其核心思想是：**每个附加层都应该更容易地包含原始函数作为其元素之一。**

#### 残差块
在残差块中，输入可通过跨层数据线路更快地向前传播。
![](https://zh-v2.d2l.ai/_images/residual-block.svg)

ResNet沿用了VGG完整的$(3\times 3)$卷积层设计。 残差块里:
* 首先有2个有相同输出通道数的$(3\times 3)$卷积层。 每个卷积层后接一个批量规范化层和ReLU激活函数。
* 然后通过跨层数据通路，跳过这2个卷积运算，将输入直接加在最后的ReLU激活函数前。
* 这样的设计要求2个卷积层的输出与输入**形状一样**，从而使它们可以相加。 如果想改变通道数，就需要引入一个额外的$(1\times 1)$卷积层来将输入变换成需要的形状后再做相加运算。


残差块的实现如下：

In [6]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

此代码生成两种类型的网络：
1. 当`use_1x1conv=False`时，应用ReLU非线性函数之前，将输入添加到输出。
2. 当`use_1x1conv=True`时，添加通过$(1 \times 1)$卷积调整通道和分辨率。

下面观察一下输出:

In [7]:
blk = Residual(3,3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

我们也可以在增加输出通道数的同时，减半输出的高和宽。

In [8]:
blk = Residual(3,6, use_1x1conv=True, strides=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

#### ResNet网络
> 注意：ResNet每个卷积层后增加了批量规范化层

* 使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。
* 第一个模块的通道数同输入通道数一致。
* 由于之前已经使用了步幅为2的最大汇聚层，所以无须减小高和宽。
* 之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

In [9]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [10]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

接着在ResNet加入所有残差块，这里每个模块使用2个残差块。

In [11]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

In [12]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

![](https://zh-v2.d2l.ai/_images/resnet18.svg)

观察各层的输出：

In [13]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])
